In [2]:
!python -m venv venv

In [3]:
!venv\Scripts\activate

In [ ]:
%pip install pandas scikit-learn
%pip install pyodbc

In [ ]:
# DRIVER={ODBC Driver 17 for SQL Server};SERVER=REVISION-PC;DATABASE=CQ_BI_13_DDS;Trusted_connection=yes

In [42]:
import pandas as pd
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=REVISION-PC;DATABASE=CQ_BI_13_DDS;Trusted_connection=yes'
conn = pyodbc.connect(connection_string)

sql = """
    SELECT
    fp.total_sold_products,
    fp.branch_id,
    fp.customer_type_id,
    fp.payment_type_id,
    fp.datetime_id,
    dp.payment_type_name,
    dct.customer_type_name,
    dpr.unit_price,  -- Assuming 'unit_price' is in the 'dim_product' table
    db.city_name,
    dd.day_number,
    dm.month_number,
    dy.year_number
FROM fact_purchase fp
JOIN dim_payment_type dp ON fp.payment_type_id = dp.payment_type_id
JOIN dim_customer_type dct ON fp.customer_type_id = dct.customer_type_id
JOIN dim_product dpr ON fp.product_id = dpr.product_id  -- Join with dim_product to get 'unit_price'
JOIN dim_branch db ON fp.branch_id = db.branch_id
JOIN dim_datetime ddt ON fp.datetime_id = ddt.datetime_id
JOIN dim_day dd ON ddt.day_id = dd.day_id
JOIN dim_month dm ON dd.month_id = dm.month_id
JOIN dim_year dy ON dm.year_id = dy.year_id
"""
df = pd.read_sql(sql, conn)
label_encoder = LabelEncoder()
df['payment_type_name'] = label_encoder.fit_transform(df['payment_type_name'])
df['customer_type_name'] = label_encoder.fit_transform(df['customer_type_name'])
df['city_name'] = label_encoder.fit_transform(df['city_name'])
X = df.drop('total_sold_products', axis=1)
y = df['total_sold_products']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')
new_data = pd.DataFrame({
    'branch_id': [1],
    'customer_type_id': [2],
    'payment_type_id': [3],
    'datetime_id': [4],
    'payment_type_name': ['Credit card'],
    'customer_type_name': ['Member'],
    'unit_price': [10.5],
    'city_name': ['CityA'],
    'day_number': [15],
    'month_number': [6],
    'year_number': [2022]
})
new_data['payment_type_name'] = label_encoder.transform(new_data['payment_type_name'])
new_data['customer_type_name'] = label_encoder.transform(new_data['customer_type_name'])
new_data['city_name'] = label_encoder.transform(new_data['city_name'])
new_predictions = model.predict(new_data)
print(f'Predicted total_sold_products: {new_predictions[0]}')

C:\Users\Thuan\AppData\Local\Temp\ipykernel_17044\3677594992.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


Mean Squared Error: 10.287367487684731


ValueError: y contains previously unseen labels: 'Cash'